In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineTempView").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 11:22:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in data. 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...|2015-08-31 00:00:00|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...| 

In [4]:
# Create a temporary view.   
music_df.createOrReplaceTempView('music')

In [5]:
# Using sql return the first 10 product titles that have a 5 star rating. 
spark.sql("SELECT product_title FROM music WHERE star_rating==5 LIMIT 10").show()

+--------------------+
|       product_title|
+--------------------+
|Whatever's for Us...|
|Same Trailer Diff...|
| Soaring (Jazz Club)|
|         Pain Killer|
|A Thoughtiverse U...|
|Comfort of Her Wings|
|Sleep Soundly CD ...|
|          Blurryface|
|   Come Away with Me|
| Tomorrow Is My Turn|
+--------------------+



In [6]:
# Return the product title and review comments (review_body) for products that had a 1 star rating.
spark.sql("SELECT product_title, review_body FROM music WHERE star_rating==1").show()

+--------------------+--------------------+
|       product_title|         review_body|
+--------------------+--------------------+
|Seems Like Old Times|Was looking for s...|
|               Venom|i have 5 other im...|
|Symphony 9 La Gra...|36 minutes for th...|
|              Kismet|         NOT SO GOOD|
|#1 Cool Dance Bea...|I didn't understa...|
|Karaoke - Johnny ...|It came with NO l...|
|           It's Time|Not as good as I ...|
| Catchy Math Jingles|Will never buy th...|
|Bach: Six Cello S...|I've heard this m...|
|Greatest Hits: De...|Cd was broken an ...|
|Nursery Rhymes Fl...|Flimsy pieces, wo...|
|Enigma 3: Le Roi ...|The item arrived ...|
| Illinois Blues 1973|Sound is very mud...|
|Music From The Se...|Way overpriced.  ...|
|     Burning Bridges|It's time to wrap...|
|New Wave Xmas: Ju...|I have all the Ju...|
|Still [2 CD][Delu...|Didn't like at al...|
|            Electric|Sucks! This is on...|
|Una Leyenda Flamenca|Very Good . I wil...|
|              SUCKER|When I bou

In [7]:
# Return the customer_id and the number of time the customer made a review. 
# Group the results by the customer_id and order the results by number of times the customer made a review in descending order.

query = """(SELECT customer_id, count(customer_id)
FROM music
GROUP BY customer_id
ORDER BY count(customer_id) DESC)"""
spark.sql(query).show()


+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   50736950|              7168|
|   38214553|              5412|
|   51184997|              5369|
|   18116317|              4222|
|   23267387|              4023|
|   50345651|              3793|
|   14539589|              2896|
|   15725862|              2842|
|   19380211|              2592|
|   20018062|              2568|
|   51381678|              2457|
|   37455882|              2370|
|   47924228|              2352|
|   50441674|              2282|
|   29791894|              2246|
|   22073263|              2213|
|   34639163|              2179|
|   25527589|              2128|
|   47423754|              2120|
|   50503261|              2119|
+-----------+------------------+
only showing top 20 rows

